# Bert Pre-Training from scratch 

## Introduction

## Check for GPU and libraries:

In [10]:
!rocm-smi



========================================= ROCm System Management Interface =========================================
=================================================== Concise Info ===================================================
Device  Node  IDs              Temp    Power  Partitions          SCLK     MCLK     Fan  Perf  PwrCap  VRAM%  GPU%  
              (DID,     GUID)  (Edge)  (Avg)  (Mem, Compute, ID)                                                    
0       4     0x740c,   57586  37.0°C  95.0W  N/A, N/A, 0         800Mhz   1600Mhz  0%   auto  560.0W  5%     0%    
1       5     0x740c,   45873  39.0°C  N/A    N/A, N/A, 0         1700Mhz  1600Mhz  0%   auto  0.0W    5%     9%    
2       2     0x740c,   14571  40.0°C  92.0W  N/A, N/A, 0         800Mhz   1600Mhz  0%   auto  560.0W  7%     0%    
3       3     0x740c,   27432  35.0°C  N/A    N/A, N/A, 0         800Mhz   1600Mhz  0%   auto  0.0W    6%     0%    
4       8     0x740c,   30939  33.0°C  86.0W  N/A, N/A, 0     

In [11]:
!lscpu 

Architecture:            x86_64
  CPU op-mode(s):        32-bit, 64-bit
  Address sizes:         48 bits physical, 48 bits virtual
  Byte Order:            Little Endian
CPU(s):                  128
  On-line CPU(s) list:   0-127
Vendor ID:               AuthenticAMD
  Model name:            AMD EPYC 7763 64-Core Processor
    CPU family:          25
    Model:               1
    Thread(s) per core:  1
    Core(s) per socket:  64
    Socket(s):           2
    Stepping:            1
    Frequency boost:     enabled
    CPU max MHz:         2450.0000
    CPU min MHz:         1500.0000
    BogoMIPS:            4890.91
    Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mc
                         a cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall n
                         x mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep_go
                         od nopl nonstop_tsc cpuid extd_apicid aperfmperf rapl p
                         ni pclmulqdq monitor ssse3

In [12]:
import torch

# check enabled GPU
print(torch.cuda.is_available())
for i in range(torch.cuda.device_count()):
   print(torch.cuda.get_device_properties(i))

True
_CudaDeviceProperties(name='AMD Instinct MI250X/MI250', major=9, minor=0, gcnArchName='gfx90a:sramecc+:xnack-', total_memory=65520MB, multi_processor_count=104, uuid=63323962-3731-6162-6164-626232366438, L2_cache_size=8MB)
_CudaDeviceProperties(name='AMD Instinct MI250X/MI250', major=9, minor=0, gcnArchName='gfx90a:sramecc+:xnack-', total_memory=65520MB, multi_processor_count=104, uuid=61323937-6262-3735-3131-373961323732, L2_cache_size=8MB)
_CudaDeviceProperties(name='AMD Instinct MI250X/MI250', major=9, minor=0, gcnArchName='gfx90a:sramecc+:xnack-', total_memory=65520MB, multi_processor_count=104, uuid=36653564-3266-6336-3333-373563623430, L2_cache_size=8MB)
_CudaDeviceProperties(name='AMD Instinct MI250X/MI250', major=9, minor=0, gcnArchName='gfx90a:sramecc+:xnack-', total_memory=65520MB, multi_processor_count=104, uuid=36663136-3964-3862-6463-366333336561, L2_cache_size=8MB)
_CudaDeviceProperties(name='AMD Instinct MI250X/MI250', major=9, minor=0, gcnArchName='gfx90a:sramecc+:

In [13]:
from tqdm import tqdm
from multiprocessing import cpu_count
from datasets import load_dataset

# set cache directory out of $HOME to $WORK
import os
os.environ["HF_HOME"] = "cache/"
default_cache_dir = "cache/"

import transformers 
print(transformers.__version__)

4.48.1


## Dataset 

### Wikipedia pages (2023) (Portuguese)

- each document (page in wikipedia) is saved in a row 

- the wikipedia dataset has several columns, including title and etc. I only used the main text of the page

- downloading dataset from hugging face only needs to be done once, and it will be saved in cache for future loads!

https://huggingface.co/datasets/wikimedia/wikipedia

In [14]:
wikipedia = load_dataset("wikimedia/wikipedia", 
                         "20231101.pt", 
                         split="train", 
                         num_proc=cpu_count(),
                         cache_dir=default_cache_dir
                         )

wikipedia = wikipedia.remove_columns([col for col in wikipedia.column_names if col != "text"])  # only keep the 'text' column

print(wikipedia, '\n')

print(wikipedia[0]["text"][:1000])

Dataset({
    features: ['text'],
    num_rows: 1112246
}) 

Astronomia é uma ciência natural que estuda corpos celestes (como estrelas, planetas, cometas, nebulosas, aglomerados de estrelas, galáxias) e fenômenos que se originam fora da atmosfera da Terra (como a radiação cósmica de fundo em micro-ondas). Preocupada com a evolução, a física e a química de objetos celestes, bem como a formação e o desenvolvimento do universo.

A astronomia é uma das mais antigas ciências. Culturas pré-históricas deixaram registrados vários artefatos astronômicos, como Stonehenge, os montes de Newgrange e os menires. As primeiras civilizações, como os babilônios, gregos, chineses, indianos, persas e maias realizaram observações metódicas do céu noturno. No entanto, a invenção do telescópio permitiu o desenvolvimento da astronomia moderna. Historicamente, a astronomia incluiu disciplinas tão diversas como astrometria, navegação astronômica, astronomia observacional e a elaboração de calendários. Durante 

BrWac 

https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC

The BrWaC (Brazilian Portuguese Web as Corpus) is a large corpus constructed following the Wacky framework,
which was made public for research purposes. The current corpus version, released in January 2017, is composed by
3.53 million documents, 2.68 billion tokens and 5.79 million types. Please note that this resource is available
solely for academic research purposes, and you agreed not to use it for any commercial applications.

In [15]:
brwac = load_dataset("brwac", 
                    data_dir="dataset-brwac",
                    split="train", 
                    num_proc=cpu_count(),
                    cache_dir=default_cache_dir,
                    trust_remote_code=True
                    )

brwac = brwac.remove_columns([col for col in brwac.column_names if col != "text"])  # only keep the 'text' column

brwac

Loading dataset shards:   0%|          | 0/38 [00:00<?, ?it/s]

Dataset({
    features: ['text'],
    num_rows: 3530796
})

In [16]:
brwac[0]["text"]["paragraphs"]

[['Conteúdo recente'],
 ['ESPUMA MARROM CHAMADA "NINGUÉM MERECE"'],
 ['31 de Agosto de 2015, 7:07 , por paulo soavinski - | No one following this article yet.'],
 ['Visualizado 202 vezes'],
 ['JORNAL ELETRÔNICO DA ILHA DO MEL'],
 ['Uma espuma marrom escuro tem aparecido com frequência na Praia de Fora.',
  'Na faixa de areia ela aparece disseminada e não chama muito a atenção.',
  'No Buraco do Aipo, com muitas pedras, ela aparece concentrada.',
  'É fácil saber que esta espuma estranha está lá, quando venta.',
  'Pequenos algodões de espuma começam a flutuar no espaço, pertinho da Praia do Saquinho.',
  'Quem pode ajudar na coleta deste material, envio a laboratório renomado e pagamento de análises, favor entrar em contato com o site.']]

In [17]:
from bs4 import BeautifulSoup
from ftfy import fix_text

for para in brwac[0]["text"]["paragraphs"]:
    for doc in para:
        soup = BeautifulSoup(doc, 'html.parser')
        print( fix_text(soup.get_text())  )

Conteúdo recente
ESPUMA MARROM CHAMADA "NINGUÉM MERECE"
31 de Agosto de 2015, 7:07 , por paulo soavinski - | No one following this article yet.
Visualizado 202 vezes
JORNAL ELETRÔNICO DA ILHA DO MEL
Uma espuma marrom escuro tem aparecido com frequência na Praia de Fora.
Na faixa de areia ela aparece disseminada e não chama muito a atenção.
No Buraco do Aipo, com muitas pedras, ela aparece concentrada.
É fácil saber que esta espuma estranha está lá, quando venta.
Pequenos algodões de espuma começam a flutuar no espaço, pertinho da Praia do Saquinho.
Quem pode ajudar na coleta deste material, envio a laboratório renomado e pagamento de análises, favor entrar em contato com o site.


In [18]:
def para2doc(example):
    list_doc = []

    for row in example["text"]:
        
        str_doc = ""
        
        for para in row["paragraphs"]:

            for doc in para:
                
                soup = BeautifulSoup(doc, 'html.parser') 

                str_doc += fix_text(soup.get_text()) + '\n'

        list_doc.append(str_doc)

    return {"text" : list_doc}

preprocessed_brwac = brwac.map(para2doc,
                                batched = True,
                                remove_columns=["text"],
                                num_proc = cpu_count(),
                                )

# preprocessed_brwac = preprocessed_brwac.rename_column("paragraphs", "text")

preprocessed_brwac

Dataset({
    features: ['text'],
    num_rows: 3530796
})

In [19]:
preprocessed_brwac[0]["text"]

'Conteúdo recente\nESPUMA MARROM CHAMADA "NINGUÉM MERECE"\n31 de Agosto de 2015, 7:07 , por paulo soavinski - | No one following this article yet.\nVisualizado 202 vezes\nJORNAL ELETRÔNICO DA ILHA DO MEL\nUma espuma marrom escuro tem aparecido com frequência na Praia de Fora.\nNa faixa de areia ela aparece disseminada e não chama muito a atenção.\nNo Buraco do Aipo, com muitas pedras, ela aparece concentrada.\nÉ fácil saber que esta espuma estranha está lá, quando venta.\nPequenos algodões de espuma começam a flutuar no espaço, pertinho da Praia do Saquinho.\nQuem pode ajudar na coleta deste material, envio a laboratório renomado e pagamento de análises, favor entrar em contato com o site.\n'

### C4 (2020) (Portuguese)

https://huggingface.co/datasets/allenai/c4

https://paperswithcode.com/dataset/c4

### Oscar

- noisy, avoid for now


### MC4 (legal)

- multilingual variant of the C4 dataset, composed of natural text drawn from the public Common Crawl Web Scrape.

- consider cleaning more this dataset, it is noisy

https://huggingface.co/datasets/joelniklaus/legal-mc4

### Multi_Legal_Pile (pt)

- is a large-scale multilingual legal dataset suited for pretraining language models. It spans over 24 languages and five legal text types.

- case_law dataset (Jurisprudência)

- very huge dataset, used only 5 %

https://huggingface.co/datasets/joelniklaus/Multi_Legal_Pile

### Lener-br dataset

- LeNER-Br is a dataset for named entity recognition (NER) in Brazilian Legal Text.

- Composed of 70 legal documents, manually curated and tagged

https://github.com/peluz/lener-br/


### Concatenate Datasets 

- skip mc4 for now (noisy)

- lener is very small compared with the others

- complete dataset is around 50% wikipedia, 50% legal docs

- 1.97 M documents

In [20]:
from datasets import concatenate_datasets

# raw_datasets = concatenate_datasets([wikipedia, brwac])
raw_datasets = concatenate_datasets([wikipedia, preprocessed_brwac])

raw_datasets

Dataset({
    features: ['text'],
    num_rows: 4643042
})

In [21]:
import nltk

stopwords = nltk.corpus.stopwords.words('portuguese')

stopwords

['a',
 'à',
 'ao',
 'aos',
 'aquela',
 'aquelas',
 'aquele',
 'aqueles',
 'aquilo',
 'as',
 'às',
 'até',
 'com',
 'como',
 'da',
 'das',
 'de',
 'dela',
 'delas',
 'dele',
 'deles',
 'depois',
 'do',
 'dos',
 'e',
 'é',
 'ela',
 'elas',
 'ele',
 'eles',
 'em',
 'entre',
 'era',
 'eram',
 'éramos',
 'essa',
 'essas',
 'esse',
 'esses',
 'esta',
 'está',
 'estamos',
 'estão',
 'estar',
 'estas',
 'estava',
 'estavam',
 'estávamos',
 'este',
 'esteja',
 'estejam',
 'estejamos',
 'estes',
 'esteve',
 'estive',
 'estivemos',
 'estiver',
 'estivera',
 'estiveram',
 'estivéramos',
 'estiverem',
 'estivermos',
 'estivesse',
 'estivessem',
 'estivéssemos',
 'estou',
 'eu',
 'foi',
 'fomos',
 'for',
 'fora',
 'foram',
 'fôramos',
 'forem',
 'formos',
 'fosse',
 'fossem',
 'fôssemos',
 'fui',
 'há',
 'haja',
 'hajam',
 'hajamos',
 'hão',
 'havemos',
 'haver',
 'hei',
 'houve',
 'houvemos',
 'houver',
 'houvera',
 'houverá',
 'houveram',
 'houvéramos',
 'houverão',
 'houverei',
 'houverem',
 'hou

### Data preprocessing

Split Dataset documents by their paragraphs, to generate more rows

Each page will be split by their paragraphs, each one becoming a new document

In [22]:
def analize_texts(row):

    list_para = []
    list_words = []
    list_stopwords = []
    list_average = []

    for doc in row["text"]:
        for paragraph in doc.split('\n'):

            # strip whitespaces
            paragraph = paragraph.strip()

            # skip single or empty worded paragraphs
            if (len(paragraph.split()) < 2):
                continue

            # count how many stopwords are in the paragraph
            stopwords_cnt = 0    
            for word in paragraph.split():
                for stop in stopwords:
                    if stop.casefold() == word.casefold():  # insensitive case
                        stopwords_cnt += 1
                        break # count once and speed up everything
            
            # count non whitespace characters
            characters = 0
            for word in paragraph.split():
                characters += len(word)

            list_para.append(paragraph)
            list_words.append(len(paragraph.split()))
            list_stopwords.append(stopwords_cnt)
            list_average.append(characters/len(paragraph.split()))

    return {"paragraphs" : list_para, "num_words" : list_words, "stopwords" : list_stopwords, "average" : list_average}

preprocessed_datasets = raw_datasets.map(analize_texts,
                                         batched = True,
                                         remove_columns=["text"],
                                         num_proc = cpu_count(),
                                        )

preprocessed_datasets = preprocessed_datasets.rename_column("paragraphs", "text")

preprocessed_datasets

Dataset({
    features: ['text', 'num_words', 'stopwords', 'average'],
    num_rows: 156783923
})

In [23]:
preprocessed_datasets.save_to_disk("dataset/para-num-stop-avg")

Saving the dataset (0/49 shards):   0%|          | 0/156783923 [00:00<?, ? examples/s]

- Remove documents with more than 512 words (will be truncated and lose meaning)

- also, the relative amount is insignificant

----

- Remove text that has less than 20 words. 

- This will save up a lot of computational effort, while keeping learning still good

In [13]:
preprocessed_name = "dataset/split-by-paragraph"

In [14]:
from datasets import load_from_disk

preprocessed_datasets2 = load_from_disk(preprocessed_name)

Loading dataset from disk:   0%|          | 0/17 [00:00<?, ?it/s]

Separate dataset into train and test

*The splits are shuffled by default, but you can set shuffle=False to prevent shuffling.*

In [15]:
split_dataset = preprocessed_datasets2.train_test_split(test_size=0.1, shuffle=True, seed=42)

split_dataset 

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 56726693
    })
    test: Dataset({
        features: ['text'],
        num_rows: 6302966
    })
})

In [16]:
split_name_dataset = "dataset/split-by-paragraph-train-test"

save

load

In [17]:
from datasets import load_from_disk

split_dataset = load_from_disk(split_name_dataset)

Loading dataset from disk:   0%|          | 0/26 [00:00<?, ?it/s]

In [18]:
split_dataset["train"][:5]["text"]

['Ao final de maio deste ano, quase 100 pessoas foram detidas numa investida global contra os criadores, vendedores e usuários do Blackshades RAT.',
 'Para celebrar este acontecimento, realizou-se uma Celebração Eucarística na Catedral Metropolitana de Pelotas com a presença de todas as crianças, familiares, educadores, voluntários, irmãs, diretoria, amigos e amigas que vieram celebrar conosco.',
 '- (OFF) O Rei continua.Assim Sagínero Jeiper representa o começo da vida do ser que queira conquistar o Auto-equilibrio da Vida Espiritual até mesmo antes de iniciar-se como um discípulo de um mestre físico.- E assim Sagínero se expõe com sua consciência dizendo com ênfase.',
 'Apesar de a identidade do suspeito ser conhecida, a Polícia pediu sigilo até a divulgação do resultado da perícia efetuada no computador do suspeito.',
 'Na manhã do dia 25 de Setembro de 1968, Américo Boavida foi vitimado por um bombardeamento aéreo do exército português à "Base Hanói II" do MPLA, onde se encontrava,

In [19]:
split_dataset["test"][:5]["text"]

['Mais do que isso, é a intenção da Sete Engenharia, que executa obras no local, próximo à Praça da Bíblia.',
 'Ele gostou da idéia, mas desistiu assim que um colega disse que era estranho, pois só pintava os desenhos de preto.',
 'O objetivo geral deste trabalho foi verificar quais os procedimentos adotados para a indexação da informação jurídica e como isso poderia ser refletido em uma política de indexação.',
 'Não encontro palavras que expressem o significado em minha vida do aprendizado do canto orfeônico e da participação em corais, desde criança até a universidade (fui a primeira inscrita no Coral da UFMA).',
 'Finalmente, o Papa Bento XVI invoca a bênção de Deus sobre a FAO e sublinha que "a Igreja está sempre pronta para trabalhar pela derrota da fome.']

## Tokenizer 

In [ ]:
vocabulary_size = 32_768
context_size = 512
tokenizer_name = f"tokenizers/custom/{vocabulary_size:_}"

### Tokenizer Training

Train a tokenizer, from scratch! 

- it is important to train a tokenizer, because it is the responsible for representing the input data so the model is able to interact and interpret the vocabulary provided!

- each word used in the input, should have a token (or a sequence of tokens) that will be used to represent it, and become embeddings when interacting with the model

- the pre-loaded tokenizer is uncased, meaning that all uppercase letters will be converted to lowercase to reduce complexity and vocabulary size

It will contain the following special tokens:

    0. [PAD]: Padding (to fill empty spots)
    1. [UNK]: Unknown
    2. [CLS]: Classification (initial token used as classifier)
    3. [SEP]: Separator (for sequences) (and also it is the ending token)
    4. [MASK]: Masking (token that represents a masked token)

- It's important for a tokenizer to have a **good compression rate**, meaning that the sequence of tokens generated will be as small as possible. As we know that is computationally expensive to increase the context size (sequence token limit)

- Vocabulary size will help with the compression rate (low fertility), the higher the vocab size, the lower the need for splitting words into subtokens. 

#### Specials Tokens

In [22]:
# ordered by the index
custom_special_tokens = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"]

#### Normalizer

In [23]:
from tokenizers.normalizers import NFC, Lowercase, Replace
from tokenizers import normalizers

# replace latex usage of "aspas"
# NFC - Canonico (evita equivalencia) e composto, menos characteres
# lowercase - unifica maiuscula e mi nusculo, que geralmente muda pouco (o positional embedding pode cuidar de comecar com maiuscula)

# NFC, NFK
# Ã A~

custom_normalizer = normalizers.Sequence([
    Replace("``", '"'), 
    Replace("''", '"'),
    NFC(), 
    Lowercase(),
])

print("Å".encode("utf-8"), "Å".encode("utf-8"))

normalized_sample = custom_normalizer.normalize_str("Oi, Å Å tudo BEM? O ``quê'' \"essa\" ''função'' faz?")

print(normalized_sample.encode("utf-8"))

normalized_sample

b'\xc3\x85' b'\xe2\x84\xab'
b'oi, \xc3\xa5 \xc3\xa5 tudo bem? o "qu\xc3\xaa" "essa" "fun\xc3\xa7\xc3\xa3o" faz?'


'oi, å å tudo bem? o "quê" "essa" "função" faz?'

#### Pre-Tokenizer

In [24]:
from tokenizers import pre_tokenizers
from tokenizers.pre_tokenizers import Punctuation, Metaspace, WhitespaceSplit, Digits
# to keep the original words (separed by whitespaces), we use metaspaces, represented by:
# “▁” U+2581 Lower One Eighth Block Unicode Character

# Digits(individual_digits=True), Whitespace()

custom_pre_tokenizer = pre_tokenizers.Sequence([WhitespaceSplit(), Punctuation(), Digits(individual_digits=False), Metaspace(replacement="▁", prepend_scheme="always")])

custom_pre_tokenizer.pre_tokenize_str("Quer saber, vamos ver o que esse pre-tokenizer faz com numeros: 12345 123 192 193!?!")

[('▁Quer', (0, 4)),
 ('▁saber', (5, 10)),
 ('▁,', (10, 11)),
 ('▁vamos', (12, 17)),
 ('▁ver', (18, 21)),
 ('▁o', (22, 23)),
 ('▁que', (24, 27)),
 ('▁esse', (28, 32)),
 ('▁pre', (33, 36)),
 ('▁-', (36, 37)),
 ('▁tokenizer', (37, 46)),
 ('▁faz', (47, 50)),
 ('▁com', (51, 54)),
 ('▁numeros', (55, 62)),
 ('▁:', (62, 63)),
 ('▁12345', (64, 69)),
 ('▁123', (70, 73)),
 ('▁192', (74, 77)),
 ('▁193', (78, 81)),
 ('▁!', (81, 82)),
 ('▁?', (82, 83)),
 ('▁!', (83, 84))]

In [25]:
custom_pre_tokenizer.pre_tokenize_str("o quê ele faz \t com         espaços adicionais \n\n e quebras de linhas?   ")

[('▁o', (0, 1)),
 ('▁quê', (2, 5)),
 ('▁ele', (6, 9)),
 ('▁faz', (10, 13)),
 ('▁com', (16, 19)),
 ('▁espaços', (28, 35)),
 ('▁adicionais', (36, 46)),
 ('▁e', (50, 51)),
 ('▁quebras', (52, 59)),
 ('▁de', (60, 62)),
 ('▁linhas', (63, 69)),
 ('▁?', (69, 70))]

#### Post-Tokenizer

In [26]:
from tokenizers.processors import TemplateProcessing

custom_post_processor = TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[("[CLS]", custom_special_tokens.index("[CLS]")), ("[SEP]", custom_special_tokens.index("[SEP]"))],
)

#### Decoder

In [27]:
from tokenizers.decoders import Metaspace

custom_decoder = Metaspace()

#### Train tokenizer with unigram model

In [28]:
from tokenizers import Tokenizer
from tokenizers.models import Unigram

custom_tokenizer = Tokenizer(Unigram())

custom_tokenizer.normalizer = custom_normalizer
custom_tokenizer.pre_tokenizer = custom_pre_tokenizer
custom_tokenizer.post_processor = custom_post_processor
custom_tokenizer.decoder = custom_decoder

#### Convert to a fast instance of tokenizer

In [29]:
from transformers import PreTrainedTokenizerFast

fast_custom_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=custom_tokenizer,
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    bos_token="[CLS]",
    sep_token="[SEP]",
    eos_token="[SEP]",
    mask_token="[MASK]",
    padding_side="right",
)

Save the tokenizer: 

### Load Tokenizer

In [30]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, 
                                          local_files_only=True, 
                                          cache_dir = default_cache_dir
                                          )

tokenizer

PreTrainedTokenizerFast(name_or_path='tokenizers/custom/32_768', vocab_size=32768, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

### Test tokenizer

In [31]:
# print( tokenizer_albert.convert_ids_to_tokens( tokenizer_albert.encode("Olá pessoal, Como vocês estão 😁 ?") ) )

print( tokenizer.convert_ids_to_tokens( tokenizer.encode("Olá pessoal, Como vocês estão 😁 ?") ) )

['[CLS]', '▁olá', '▁pessoal', '▁', ',', '▁como', '▁vocês', '▁estão', '▁', '😁', '▁', '?', '[SEP]']


In [32]:
def test_tokenizer(sample):
    encoding = tokenizer.encode(sample)
    print(encoding)
    print(tokenizer.convert_ids_to_tokens(encoding))
    print()

test_tokenizer('''
Atiraram o pau no gato, mas o gato não morreu
''')

test_tokenizer('''
Não sei, só sei que foi assim...
''')

test_tokenizer('''
Testando o modo continuo, e tambem o modo subjuntivo ( soubesse )
''')

test_tokenizer('''
justo, justa, justiça, injusto, injustamente, justamente, junto
''')

test_tokenizer('''
    testando acentos, será que manter os acentos melhora a acurácia do meu modelo? 
''')

test_tokenizer('''
    amigo: amiguinho, amiga, amiguinha, amigão, amigaço, amigalhaço
''')

test_tokenizer(
    preprocessed_datasets2[0]["text"]
)

[2, 5, 7, 16208, 5, 8, 2535, 24, 5553, 5, 6, 47, 5, 8, 5553, 32, 2151, 3]
['[CLS]', '▁', 'a', 'tiraram', '▁', 'o', '▁pau', '▁no', '▁gato', '▁', ',', '▁mas', '▁', 'o', '▁gato', '▁não', '▁morreu', '[SEP]']

[2, 32, 774, 5, 6, 104, 774, 13, 43, 113, 5, 12, 5, 12, 5, 12, 3]
['[CLS]', '▁não', '▁sei', '▁', ',', '▁só', '▁sei', '▁que', '▁foi', '▁assim', '▁', '.', '▁', '.', '▁', '.', '[SEP]']

[2, 5, 16436, 5, 8, 427, 12153, 5, 6, 5, 11, 6102, 5, 8, 427, 587, 10929, 1272, 5, 31, 13588, 5, 30, 3]
['[CLS]', '▁', 'testando', '▁', 'o', '▁modo', '▁continuo', '▁', ',', '▁', 'e', '▁tambem', '▁', 'o', '▁modo', '▁sub', 'jun', 'tivo', '▁', '(', '▁soubesse', '▁', ')', '[SEP]']

[2, 3563, 5, 6, 4058, 5, 6, 378, 5, 6, 13011, 5, 6, 5, 19156, 5, 6, 2229, 5, 6, 480, 3]
['[CLS]', '▁justo', '▁', ',', '▁justa', '▁', ',', '▁justiça', '▁', ',', '▁injusto', '▁', ',', '▁', 'injustamente', '▁', ',', '▁justamente', '▁', ',', '▁junto', '[SEP]']

[2, 5, 16436, 5, 7, 2211, 8, 10, 5, 6, 178, 13, 644, 5, 8, 10, 5, 7, 2211, 

#### Objetively Evaluate Tokenizer on Compression Rate (1/Fertility)

In [33]:
def normalize_and_pre_tokenize(text):
    normalized = tokenizer.backend_tokenizer.normalizer.normalize_str(text)
    processed = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(normalized)
    return processed

normalize_and_pre_tokenize( split_dataset["train"][0]["text"] )

[('▁ao', (0, 2)),
 ('▁final', (3, 8)),
 ('▁de', (9, 11)),
 ('▁maio', (12, 16)),
 ('▁deste', (17, 22)),
 ('▁ano', (23, 26)),
 ('▁,', (26, 27)),
 ('▁quase', (28, 33)),
 ('▁100', (34, 37)),
 ('▁pessoas', (38, 45)),
 ('▁foram', (46, 51)),
 ('▁detidas', (52, 59)),
 ('▁numa', (60, 64)),
 ('▁investida', (65, 74)),
 ('▁global', (75, 81)),
 ('▁contra', (82, 88)),
 ('▁os', (89, 91)),
 ('▁criadores', (92, 101)),
 ('▁,', (101, 102)),
 ('▁vendedores', (103, 113)),
 ('▁e', (114, 115)),
 ('▁usuários', (116, 124)),
 ('▁do', (125, 127)),
 ('▁blackshades', (128, 139)),
 ('▁rat', (140, 143)),
 ('▁.', (143, 144))]

In [34]:
def count_tokens(batch):
    
    original_tokens = 0
    generated_tokens = 0

    for doc in batch["text"]:

        original_tokens += len( normalize_and_pre_tokenize(doc) )
        generated_tokens += len( tokenizer.encode(doc) )
        
    # Add the token counts as a new column to the batch
    return {
        "generated": [generated_tokens],
        "original": [original_tokens]
    }

evaluate_fertility = split_dataset["train"].map(count_tokens, 
                                      batched=True,
                                      remove_columns=["text"], 
                                      num_proc=cpu_count()
                                      )

In [35]:
total_generated = sum(evaluate_fertility["generated"])
total_original = sum(evaluate_fertility["original"])

print(total_generated, total_original)

print("fertility:", total_generated/total_original)

3668402771 2388707891
fertility: 1.5357268190143891


## Tokenize the dataset

In [36]:
tokenized_datasets_name = f"dataset/tokenized-for-training/custom/vocab_size:{vocabulary_size:_}/context_size:{context_size}"

tokenized_datasets_name

'dataset/tokenized-for-training/custom/vocab_size:32_768/context_size:512'

The bert-base-uncased tokenizer was configured to use a {model_max_length} of 512, meaning that the sequence input given to the bert model can have up to 512 tokens (context size).

Nevertheless, for better performance (speed), I will hard limit if to 128:

In [37]:
tokenizer.model_max_length = context_size

tokenizer

PreTrainedTokenizerFast(name_or_path='tokenizers/custom/32_768', vocab_size=32768, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

### Examples

In [38]:
tokenizer_example = tokenizer(
    preprocessed_datasets2[0]["text"],
    return_tensors="pt", # pt = pytorch
    max_length=context_size,
    truncation=True,
    padding="max_length",
    return_special_tokens_mask=True,
)

print(tokenizer_example.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask'])


In [39]:
tokenizer.convert_ids_to_tokens( tokenizer_example["input_ids"].tolist()[0] )

['[CLS]',
 '▁astronomia',
 '▁',
 'é',
 '▁uma',
 '▁ciência',
 '▁natural',
 '▁que',
 '▁estuda',
 '▁corpos',
 '▁',
 'celestes',
 '▁',
 '(',
 '▁como',
 '▁estrelas',
 '▁',
 ',',
 '▁planeta',
 's',
 '▁',
 ',',
 '▁cometa',
 's',
 '▁',
 ',',
 '▁',
 'nebulosa',
 's',
 '▁',
 ',',
 '▁',
 'aglomerado',
 's',
 '▁de',
 '▁estrelas',
 '▁',
 ',',
 '▁galáxia',
 's',
 '▁',
 ')',
 '▁',
 'e',
 '▁fenômenos',
 '▁que',
 '▁se',
 '▁origina',
 'm',
 '▁fora',
 '▁da',
 '▁atmosfera',
 '▁da',
 '▁terra',
 '▁',
 '(',
 '▁como',
 '▁',
 'a',
 '▁radiação',
 '▁cósmica',
 '▁de',
 '▁fundo',
 '▁em',
 '▁micro',
 '▁',
 '-',
 '▁ondas',
 '▁',
 ')',
 '▁',
 '.',
 '▁preocupada',
 '▁com',
 '▁',
 'a',
 '▁evolução',
 '▁',
 ',',
 '▁',
 'a',
 '▁física',
 '▁',
 'e',
 '▁',
 'a',
 '▁química',
 '▁de',
 '▁objetos',
 '▁',
 'celestes',
 '▁',
 ',',
 '▁bem',
 '▁como',
 '▁',
 'a',
 '▁formação',
 '▁',
 'e',
 '▁',
 'o',
 '▁desenvolvimento',
 '▁do',
 '▁universo',
 '▁',
 '.',
 '[SEP]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD

In [40]:
tokenizer_example["input_ids"] # the id of the token

tensor([[    2, 13599,     5,    25,    29,  1009,   820,    13,  4106,  4310,
             5, 21767,     5,    31,    36,  2936,     5,     6,  1558,    10,
             5,     6, 13428,    10,     5,     6,     5, 19696,    10,     5,
             6,     5, 14962,    10,     9,  2936,     5,     6,  8319,    10,
             5,    30,     5,    11,  6255,    13,    26,  8309,    23,   343,
            15,  4544,    15,   367,     5,    31,    36,     5,     7,  7331,
         16437,     9,   795,    16,  1811,     5,    20,  4436,     5,    30,
             5,    12,  9883,    19,     5,     7,  1645,     5,     6,     5,
             7,   932,     5,    11,     5,     7,  2981,     9,  2042,     5,
         21767,     5,     6,    87,    36,     5,     7,   494,     5,    11,
             5,     8,   247,    14,  1477,     5,    12,     3,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [41]:
tokenizer_example["token_type_ids"] # 0 for sentenceA and special tokens, 1 for sentenceB

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0

In [42]:
tokenizer_example["attention_mask"] # 0 for tokens that don't interfere in the attention computation (value matrix), 1 otherwise

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0

In [43]:
tokenizer_example["special_tokens_mask"] # 1 for special tokens

tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1

### Process the Dataset:

This next step is to manipulate the dataset for a suitable format for the model:

- That is, tokenize the dataset, and truncate the documents with more than {model_max_length} tokens. 

- This is done in the CPU in batches and parellized across the cores.

Save the dataset, now, already tokenized, locally. 

### Load Tokenized Datasets

In [44]:
from datasets import load_from_disk

# load tokenized dataset locally
tokenized_datasets = load_from_disk(tokenized_datasets_name)

print(tokenized_datasets["train"].features, '\n')

tokenized_datasets

Loading dataset from disk:   0%|          | 0/409 [00:00<?, ?it/s]

Loading dataset from disk:   0%|          | 0/46 [00:00<?, ?it/s]

{'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'special_tokens_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)} 



DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask'],
        num_rows: 56726693
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask'],
        num_rows: 6302966
    })
})

## Filtering

### Filter Dataset

To train the bert model faster, we will optimize the documents used for training, that is, we will reduce the dataset to retain only good documents. 

**The criteria used**  *the documents will be filtered and only be valid when*:

    - it has more than 32 tokens.

    - it was not truncated

### Save

### Load

Make training dataset smaller for easier experimentation

In [45]:
training_dataset = tokenized_datasets["train"] #.select([i for i in range( int(1 * len(tokenized_datasets["train"])) )])

training_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask'],
    num_rows: 56726693
})

In [46]:
evaluation_dataset = tokenized_datasets["test"] #.select([i for i in range( int(1 * len(tokenized_datasets["test"])) )])

evaluation_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask'],
    num_rows: 6302966
})

convert to pytorch format and remove token type id, which is ignored by modernbert

## Model Training

import a pre-configured bert and choose a name for the model

In [47]:
model_name = f"Modern/{4.6}"

In [48]:
from transformers import ModernBertConfig

config = ModernBertConfig.from_pretrained("answerdotai/ModernBERT-base", reference_compile=False)

config

ModernBertConfig {
  "_name_or_path": "ModernBERT-base",
  "architectures": [
    "ModernBertForMaskedLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 50281,
  "classifier_activation": "gelu",
  "classifier_bias": false,
  "classifier_dropout": 0.0,
  "classifier_pooling": "mean",
  "cls_token_id": 50281,
  "decoder_bias": true,
  "deterministic_flash_attn": false,
  "embedding_dropout": 0.0,
  "eos_token_id": 50282,
  "global_attn_every_n_layers": 3,
  "global_rope_theta": 160000.0,
  "gradient_checkpointing": false,
  "hidden_activation": "gelu",
  "hidden_size": 768,
  "initializer_cutoff_factor": 2.0,
  "initializer_range": 0.02,
  "intermediate_size": 1152,
  "layer_norm_eps": 1e-05,
  "local_attention": 128,
  "local_rope_theta": 10000.0,
  "max_position_embeddings": 8192,
  "mlp_bias": false,
  "mlp_dropout": 0.0,
  "model_type": "modernbert",
  "norm_bias": false,
  "norm_eps": 1e-05,
  "num_attention_heads": 12,
  "num_hidden_layers": 22,
  "pa

In [49]:
# diminish the specs

# {bert-small} with max_position_embedding = 128

config.vocab_size = vocabulary_size
config.max_position_embeddings = 512
config.local_attention = 128
config.pad_token_id = 0
config.bos_token_id = 2
config.cls_token_id = 2
config.eos_token_id = 3
config.sep_token_id = 3

config

ModernBertConfig {
  "_name_or_path": "ModernBERT-base",
  "architectures": [
    "ModernBertForMaskedLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 2,
  "classifier_activation": "gelu",
  "classifier_bias": false,
  "classifier_dropout": 0.0,
  "classifier_pooling": "mean",
  "cls_token_id": 2,
  "decoder_bias": true,
  "deterministic_flash_attn": false,
  "embedding_dropout": 0.0,
  "eos_token_id": 3,
  "global_attn_every_n_layers": 3,
  "global_rope_theta": 160000.0,
  "gradient_checkpointing": false,
  "hidden_activation": "gelu",
  "hidden_size": 768,
  "initializer_cutoff_factor": 2.0,
  "initializer_range": 0.02,
  "intermediate_size": 1152,
  "layer_norm_eps": 1e-05,
  "local_attention": 128,
  "local_rope_theta": 10000.0,
  "max_position_embeddings": 512,
  "mlp_bias": false,
  "mlp_dropout": 0.0,
  "model_type": "modernbert",
  "norm_bias": false,
  "norm_eps": 1e-05,
  "num_attention_heads": 12,
  "num_hidden_layers": 22,
  "pad_token_id": 

Train for MLM task

In [50]:
from transformers import ModernBertForMaskedLM

model = ModernBertForMaskedLM(config=config)

print("parameters: ", model.num_parameters())

model

parameters:  136120832


ModernBertForMaskedLM(
  (model): ModernBertModel(
    (embeddings): ModernBertEmbeddings(
      (tok_embeddings): Embedding(32768, 768, padding_idx=0)
      (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (drop): Dropout(p=0.0, inplace=False)
    )
    (layers): ModuleList(
      (0): ModernBertEncoderLayer(
        (attn_norm): Identity()
        (attn): ModernBertAttention(
          (Wqkv): Linear(in_features=768, out_features=2304, bias=False)
          (rotary_emb): ModernBertUnpaddedRotaryEmbedding(dim=64, base=160000.0, scale_base=None)
          (Wo): Linear(in_features=768, out_features=768, bias=False)
          (out_drop): Identity()
        )
        (mlp_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): ModernBertMLP(
          (Wi): Linear(in_features=768, out_features=2304, bias=False)
          (act): GELUActivation()
          (drop): Dropout(p=0.0, inplace=False)
          (Wo): Linear(in_features=1152, out_features=768, 

In [51]:
#save model architecture and configurations to facilitate loading afterwards

model.save_pretrained(f"models/{model_name}")

to mask the input randomly, we will use a data collator, which will randomly tokenize 30% of the input as [MASK] 

it is a dynamical masking (such as the one used in roBERTa), meaning that each time it a document is given as the input in the model, as random porcentile will the masked. 

in static masking, all the dataset is prior masked, meaning that the masked tokens is predefinied and if you run multiple epochs, it won't make a difference. 

In [52]:
from transformers import DataCollatorForLanguageModeling

# mask 30% of the tokens
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm = True,
    mlm_probability=0.3
) 

As for the training arguments, 

- we will train the model passing through the dataset twice (epoch == 2)

- the batch size (number of inputs trained per iteration) is 128

- RoBERTa proved that bigger batches will output better results, that why we also use gradient accumulation steps 

- only after 4 {gradient_accumulation_steps}, the model will backpropagate and update parameters.

- each 1_000 iterations (steps) the model parameters will be locally saved

- the number of saves or checkpoints saved simultaneusly is 3

- it will used mixed precision of fp16, meaning that sometimes (when the precision won't affect the accuracy),
it will not use the float of 32 bits and instead use float of 16 bits, speeding up the training process

In [53]:
# import evaluate

# classification_metrics = evaluate.combine(["accuracy"])

# def compute_metrics_modern(eval_pred):

#     labels = eval_pred.label_ids
#     preds = eval_pred.predictions.argmax(-1)

#     return classification_metrics.compute(predictions=preds.flatten(), references=labels.flatten())

In [54]:
from transformers import Trainer, TrainingArguments
from transformers import get_wsd_schedule
from torch.optim import AdamW

total_steps = 500_000

training_args = TrainingArguments(
    output_dir=f'training/{model_name}',
    overwrite_output_dir=True,
    
    # num_train_epochs=1,                     # number of training epochs
    max_steps=total_steps,
    # max_steps=100,

    gradient_accumulation_steps = 1,
    # eval_accumulation_steps = 1,

    per_device_train_batch_size=32,          # batch size for training
    # per_device_eval_batch_size=32,           # batch size for evaluation

    
    logging_strategy="steps",
    logging_first_step=True, # output the initial loss
    logging_steps=1_000,
    logging_dir=f"training-logs/{model_name}",
    report_to=["tensorboard"],

    save_strategy="steps",
    save_steps=1_000,                      # Save checkpoints every 100 steps
    save_total_limit=5,                  # Limit the total number of saved checkpoints

    fp16=True,                            # Enable mixed precision for faster training

    # learning_rate=8e-4,
    # weight_decay=1e-2,
    # adam_beta1=0.9,
    # adam_beta2=0.999,
    # adam_epsilon=1e-06,
    # lr_scheduler_type=
)

# # Create default optimizer
# optimizer = AdamW(
#     model.parameters(),
#     lr = 8e-4,
#     weight_decay=1e-2,
#     betas = (0.9, 0.999),
# )

# scheduler = get_wsd_schedule(
#     AdamW,                  # Your optimizer
#     num_warmup_steps=total_steps * 0.1,   # Number of warmup steps
#     num_decay_steps=total_steps * 0.1,   # Number of decay steps
#     # num_training_steps=total_steps,  # Total number of training steps
#     num_stable_steps=total_steps * 0.8,   # Number of stable steps
#     #warmup_type="linear",   # Warmup type
#     #decay_type="1-sqrt",    # Decay type
#     min_lr_ratio=0.0,       # Minimum learning rate ratio
#     # num_cycles=0.5,         # Number of cosine cycles
# )

    # lr_scheduler_type="warmup_stable_decay",
    # lr_scheduler_kwargs = {
    #     #"optimizer": "AdamW",
    #     #"num_warmup_steps": 25,l
    #     "num_decay_steps": 25,
    #     #"num_training_steps": 1000,
    #     "num_stable_steps": 950,
    #     # "warmup_type": "linear",
    #     #"decay_type": "1-sqrt",
    #     #"min_lr_ratio": 0,
    # },
    # # warmup_steps=25,

trainer = Trainer(
    model=model,                        # Model to train
    args=training_args,                 # Training arguments
    train_dataset=training_dataset,     # Training dataset
    # eval_dataset=evaluation_dataset,    # Evaluation dataset
    data_collator=data_collator,
    # optimizers=(optimizer, scheduler),
)

In [55]:
torch.cuda.empty_cache()

In [56]:
trainer.train(resume_from_checkpoint=True)

trainer.save_model(f"trained/{model_name}")

There were missing keys in the checkpoint model loaded: ['decoder.weight'].
/work1/lgarcia/wallacelw/ModernBERT/myenv/lib64/python3.9/site-packages/transformers/trainer.py:3441: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for 

Step,Training Loss


## Testing the Model

Load the self trained model and create a pipeline that will:

- tokenize the input
 
- push to the model

- output the top 5 token that the model predict to fill each masked token

In [57]:
from transformers import pipeline, ModernBertForMaskedLM

model = ModernBertForMaskedLM.from_pretrained(f"trained/{model_name}")

model.to('cuda')

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


ModernBertForMaskedLM(
  (model): ModernBertModel(
    (embeddings): ModernBertEmbeddings(
      (tok_embeddings): Embedding(32768, 768, padding_idx=0)
      (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (drop): Dropout(p=0.0, inplace=False)
    )
    (layers): ModuleList(
      (0): ModernBertEncoderLayer(
        (attn_norm): Identity()
        (attn): ModernBertAttention(
          (Wqkv): Linear(in_features=768, out_features=2304, bias=False)
          (rotary_emb): ModernBertUnpaddedRotaryEmbedding(dim=64, base=160000.0, scale_base=None)
          (Wo): Linear(in_features=768, out_features=768, bias=False)
          (out_drop): Identity()
        )
        (mlp_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): ModernBertMLP(
          (Wi): Linear(in_features=768, out_features=2304, bias=False)
          (act): GELUActivation()
          (drop): Dropout(p=0.0, inplace=False)
          (Wo): Linear(in_features=1152, out_features=768, 

In [63]:
from transformers import Trainer, TrainingArguments
import numpy as np

evaluation_args = TrainingArguments(
    output_dir=f'evaluating/{model_name}',

    do_train=False,
    eval_strategy = "steps", 
    eval_steps=1_000,
    # eval_accumulation_steps=1,

    per_device_eval_batch_size=32,           # batch size for evaluation

    logging_strategy="steps",
    logging_steps=1_000,
    logging_dir=f'evaluation-logs/{model_name}',                # directory for storing logs
    report_to=["tensorboard"],

    save_strategy="steps",
    save_steps=1_000,                      # Save checkpoints every 100 steps
    save_total_limit=5,                  # Limit the total number of saved checkpoints

    fp16=True,    
)

evaluator = Trainer(
    model=model,   
    args=evaluation_args,
    train_dataset=[],
    eval_dataset=evaluation_dataset,         # Evaluation dataset
    data_collator=data_collator,
)


# Evaluate the model
eval_results = evaluator.evaluate()

# Print the results
print(eval_results)

/share/sw/ai/pytorch/2.5.1/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


KeyboardInterrupt: 

In [64]:
fill_mask = pipeline(
    "fill-mask",
    model=model,
    tokenizer=tokenizer
)

test1 = "Eu não entendi [MASK], como proceder."

out = tokenizer(test1)

out

Device set to use cuda:0


{'input_ids': [2, 5, 11, 18, 32, 7471, 4, 5, 6, 36, 8060, 5, 12, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [65]:
tokenizer.convert_ids_to_tokens(out['input_ids'])

['[CLS]',
 '▁',
 'e',
 'u',
 '▁não',
 '▁entendi',
 '[MASK]',
 '▁',
 ',',
 '▁como',
 '▁proceder',
 '▁',
 '.',
 '[SEP]']

In [66]:
fill_mask(test1)

[{'score': 0.1924876570701599,
  'token': 268,
  'token_str': 'nada',
  'sequence': 'eu não entendi nada , como proceder .'},
 {'score': 0.1782321184873581,
  'token': 221,
  'token_str': 'direito',
  'sequence': 'eu não entendi direito , como proceder .'},
 {'score': 0.1367182582616806,
  'token': 87,
  'token_str': 'bem',
  'sequence': 'eu não entendi bem , como proceder .'},
 {'score': 0.05346071347594261,
  'token': 164,
  'token_str': 'tudo',
  'sequence': 'eu não entendi tudo , como proceder .'},
 {'score': 0.04698919877409935,
  'token': 73,
  'token_str': 'muito',
  'sequence': 'eu não entendi muito , como proceder .'}]

In [67]:
fill_mask("Atiraram o pau no [MASK]")

[{'score': 0.1659652292728424,
  'token': 2471,
  'token_str': 'chão',
  'sequence': 'atiraram o pau no chão'},
 {'score': 0.06266004592180252,
  'token': 1599,
  'token_str': 'marido',
  'sequence': 'atiraram o pau no marido'},
 {'score': 0.0401596799492836,
  'token': 4321,
  'token_str': 'rapaz',
  'sequence': 'atiraram o pau no rapaz'},
 {'score': 0.027083080261945724,
  'token': 638,
  'token_str': 'cara',
  'sequence': 'atiraram o pau no cara'},
 {'score': 0.025046631693840027,
  'token': 11493,
  'token_str': 'rabo',
  'sequence': 'atiraram o pau no rabo'}]

In [68]:
fill_mask("Essa frase tem [MASK] tokens, portanto ele vai gerar dois [MASK]")

[[{'score': 0.25275641679763794,
   'token': 134,
   'token_str': 'dois',
   'sequence': '[CLS] essa frase tem dois tokens , portanto ele vai gerar dois[MASK][SEP]'},
  {'score': 0.13312558829784393,
   'token': 194,
   'token_str': 'três',
   'sequence': '[CLS] essa frase tem três tokens , portanto ele vai gerar dois[MASK][SEP]'},
  {'score': 0.09293114393949509,
   'token': 351,
   'token_str': 'quatro',
   'sequence': '[CLS] essa frase tem quatro tokens , portanto ele vai gerar dois[MASK][SEP]'},
  {'score': 0.07274574786424637,
   'token': 470,
   'token_str': 'cinco',
   'sequence': '[CLS] essa frase tem cinco tokens , portanto ele vai gerar dois[MASK][SEP]'},
  {'score': 0.0429421029984951,
   'token': 750,
   'token_str': 'seis',
   'sequence': '[CLS] essa frase tem seis tokens , portanto ele vai gerar dois[MASK][SEP]'}],
 [{'score': 0.040673743933439255,
   'token': 1086,
   'token_str': 'efeitos',
   'sequence': '[CLS] essa frase tem[MASK] tokens , portanto ele vai gerar dois 

## NER task

### Fine-tuning for for NER task

Download the curated dataset for portuguese NER task in the legal scope.

It already has the train, validation and test splits

In [69]:
from datasets import load_dataset

lenerNER = load_dataset("peluz/lener_br", 
                        trust_remote_code=True, 
                        num_proc=cpu_count())

lenerNER

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 7828
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1177
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1390
    })
})

In [70]:
print( lenerNER["train"][0]["id"] ) # id of the document [0, 70)
    
print( lenerNER["train"][0]["tokens"] ) # words

print( lenerNER["train"][0]["ner_tags"] ) # tags for the words

0
['EMENTA', ':', 'APELAÇÃO', 'CÍVEL', '-', 'AÇÃO', 'DE', 'INDENIZAÇÃO', 'POR', 'DANOS', 'MORAIS', '-', 'PRELIMINAR', '-', 'ARGUIDA', 'PELO', 'MINISTÉRIO', 'PÚBLICO', 'EM', 'GRAU', 'RECURSAL', '-', 'NULIDADE', '-', 'AUSÊNCIA', 'DE', 'INTERVENÇÃO', 'DO', 'PARQUET', 'NA', 'INSTÂNCIA', 'A', 'QUO', '-', 'PRESENÇA', 'DE', 'INCAPAZ', '-', 'PREJUÍZO', 'EXISTENTE', '-', 'PRELIMINAR', 'ACOLHIDA', '-', 'NULIDADE', 'RECONHECIDA', '.']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


Each ner_tag id corresponds to one of the following labels:

**BIO notation:**

B - means that is the beginning of a entity

I - means that it inside the entity (after a B tag)

O - means that this token a non-entity tokens

In [71]:
label_list = lenerNER["train"].features[f"ner_tags"].feature.names

label_list

['O',
 'B-ORGANIZACAO',
 'I-ORGANIZACAO',
 'B-PESSOA',
 'I-PESSOA',
 'B-TEMPO',
 'I-TEMPO',
 'B-LOCAL',
 'I-LOCAL',
 'B-LEGISLACAO',
 'I-LEGISLACAO',
 'B-JURISPRUDENCIA',
 'I-JURISPRUDENCIA']

Each word can be split into several tokens by our pre trained tokenizer, let's see an example:

word "PARQUET" was split into 2 tokens

In [72]:
exampleNER = lenerNER["train"][0]

example_tokenized_input = tokenizer(exampleNER["tokens"], is_split_into_words = True)

example_tokens = tokenizer.convert_ids_to_tokens(example_tokenized_input["input_ids"])

print(exampleNER["tokens"])
print(example_tokenized_input["input_ids"])
print(example_tokens)
print(exampleNER["ner_tags"])

['EMENTA', ':', 'APELAÇÃO', 'CÍVEL', '-', 'AÇÃO', 'DE', 'INDENIZAÇÃO', 'POR', 'DANOS', 'MORAIS', '-', 'PRELIMINAR', '-', 'ARGUIDA', 'PELO', 'MINISTÉRIO', 'PÚBLICO', 'EM', 'GRAU', 'RECURSAL', '-', 'NULIDADE', '-', 'AUSÊNCIA', 'DE', 'INTERVENÇÃO', 'DO', 'PARQUET', 'NA', 'INSTÂNCIA', 'A', 'QUO', '-', 'PRESENÇA', 'DE', 'INCAPAZ', '-', 'PREJUÍZO', 'EXISTENTE', '-', 'PRELIMINAR', 'ACOLHIDA', '-', 'NULIDADE', 'RECONHECIDA', '.']
[2, 16, 2091, 5, 50, 12806, 5, 7219, 5, 20, 5, 168, 9, 4873, 33, 1959, 3520, 5, 20, 8645, 5, 20, 6506, 18, 709, 53, 418, 206, 16, 1433, 17707, 5, 20, 10922, 5, 20, 1885, 9, 2463, 14, 1306, 41, 28, 5583, 5, 7, 5495, 8, 5, 20, 605, 9, 7982, 5, 20, 3566, 3580, 5, 20, 8645, 5, 7, 8247, 5, 20, 10922, 4134, 5, 12, 3]
['[CLS]', '▁em', 'enta', '▁', ':', '▁apelação', '▁', 'cível', '▁', '-', '▁', 'ação', '▁de', '▁indenização', '▁por', '▁danos', '▁morais', '▁', '-', '▁preliminar', '▁', '-', '▁arg', 'u', 'ida', '▁pelo', '▁ministério', '▁público', '▁em', '▁grau', '▁recursal', '▁',

**word_id:** the index of the word given prior to the tokenization!

check the example with word index 28

In [73]:
# if a word is split into 2 tokens, both will share the same word_id

example_word_ids = example_tokenized_input.word_ids()

for i, j in zip(example_word_ids, example_tokens):
    print(i, j)

None [CLS]
0 ▁em
0 enta
1 ▁
1 :
2 ▁apelação
3 ▁
3 cível
4 ▁
4 -
5 ▁
5 ação
6 ▁de
7 ▁indenização
8 ▁por
9 ▁danos
10 ▁morais
11 ▁
11 -
12 ▁preliminar
13 ▁
13 -
14 ▁arg
14 u
14 ida
15 ▁pelo
16 ▁ministério
17 ▁público
18 ▁em
19 ▁grau
20 ▁recursal
21 ▁
21 -
22 ▁nulidade
23 ▁
23 -
24 ▁ausência
25 ▁de
26 ▁intervenção
27 ▁do
28 ▁parque
28 t
29 ▁na
30 ▁instância
31 ▁
31 a
32 ▁qu
32 o
33 ▁
33 -
34 ▁presença
35 ▁de
36 ▁incapaz
37 ▁
37 -
38 ▁prejuízo
39 ▁existente
40 ▁
40 -
41 ▁preliminar
42 ▁
42 a
42 colhida
43 ▁
43 -
44 ▁nulidade
45 ▁reconhecida
46 ▁
46 .
None [SEP]


Tokenize the input and preprocess. 

We need also to align, because some words were split into several tokens

label of [CLS], [SEP] and for tokens that are suffixes (##) have label == -100 (meaning that it will be ignored for the ner tag prediction)

In [74]:
def tokenize_and_align_labels(examples):
    
    tokenized_inputs = tokenizer(examples["tokens"], 
                                 max_length=context_size,
                                 truncation=True, 
                                 is_split_into_words=True)

    labels = []

    for i, label in enumerate(examples[f"ner_tags"]):

        word_ids = tokenized_inputs.word_ids(batch_index=i)

        previous_word_idx = None
        label_ids = []

        for word_idx in word_ids:

            if word_idx is None: # [CLS], [SEP]
                label_ids.append(-100)

            elif word_idx != previous_word_idx: # new word
                label_ids.append(label[word_idx])

            else: # suffix of a previous token
                label_ids.append(-100)

            previous_word_idx = word_idx
            
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels

    return tokenized_inputs

tokenized_lenerNER = lenerNER.map(tokenize_and_align_labels,
                                  batched=True,
                                  num_proc=cpu_count(),
                                  )

Map (num_proc=128):   0%|          | 0/7828 [00:00<?, ? examples/s]

Map (num_proc=128):   0%|          | 0/1177 [00:00<?, ? examples/s]

Map (num_proc=128):   0%|          | 0/1390 [00:00<?, ? examples/s]

In [75]:
tokenized_lenerNER

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 7828
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1177
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1390
    })
})

In [76]:
from transformers import DataCollatorForTokenClassification

ner_collator = DataCollatorForTokenClassification(tokenizer=tokenizer,
                                                  padding=True,
                                                  )

load a evaluation metric that will be used to compute the total evaluation for a sequence (seq eval)

In [77]:
import evaluate

seqeval = evaluate.load("seqeval")

define the method to compute metrics (precision, recall, f1, accuracy)

In [78]:
import numpy as np

def compute_metrics_ner(p):

    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
}

Create dictionaries to convert id to label and vice-versa

In [79]:
id2label = {}
label2id = {}

for i, lab in enumerate(label_list):

    id2label[i] = lab
    label2id[lab] = i

print(id2label)
print(label2id)

{0: 'O', 1: 'B-ORGANIZACAO', 2: 'I-ORGANIZACAO', 3: 'B-PESSOA', 4: 'I-PESSOA', 5: 'B-TEMPO', 6: 'I-TEMPO', 7: 'B-LOCAL', 8: 'I-LOCAL', 9: 'B-LEGISLACAO', 10: 'I-LEGISLACAO', 11: 'B-JURISPRUDENCIA', 12: 'I-JURISPRUDENCIA'}
{'O': 0, 'B-ORGANIZACAO': 1, 'I-ORGANIZACAO': 2, 'B-PESSOA': 3, 'I-PESSOA': 4, 'B-TEMPO': 5, 'I-TEMPO': 6, 'B-LOCAL': 7, 'I-LOCAL': 8, 'B-LEGISLACAO': 9, 'I-LEGISLACAO': 10, 'B-JURISPRUDENCIA': 11, 'I-JURISPRUDENCIA': 12}


load the pre trained model of bert to fine tune it for NER task, attaching a Token Classification HEAD

In [80]:
from transformers import ModernBertForTokenClassification

ner_model = ModernBertForTokenClassification.from_pretrained(
    pretrained_model_name_or_path=f"trained/{model_name}",
    num_labels = len(label_list),
    id2label = id2label,
    label2id= label2id,
)

ner_model

Some weights of ModernBertForTokenClassification were not initialized from the model checkpoint at trained/Modern/4.6 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ModernBertForTokenClassification(
  (model): ModernBertModel(
    (embeddings): ModernBertEmbeddings(
      (tok_embeddings): Embedding(32768, 768, padding_idx=0)
      (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (drop): Dropout(p=0.0, inplace=False)
    )
    (layers): ModuleList(
      (0): ModernBertEncoderLayer(
        (attn_norm): Identity()
        (attn): ModernBertAttention(
          (Wqkv): Linear(in_features=768, out_features=2304, bias=False)
          (rotary_emb): ModernBertUnpaddedRotaryEmbedding(dim=64, base=160000.0, scale_base=None)
          (Wo): Linear(in_features=768, out_features=768, bias=False)
          (out_drop): Identity()
        )
        (mlp_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): ModernBertMLP(
          (Wi): Linear(in_features=768, out_features=2304, bias=False)
          (act): GELUActivation()
          (drop): Dropout(p=0.0, inplace=False)
          (Wo): Linear(in_features=1152, out_fea

Train it in batches of size 16, while also computing the evaluation for the validation set

In [81]:
from transformers import TrainingArguments

ner_training_args = TrainingArguments(
    output_dir = f"trained/NER/{model_name}",
    overwrite_output_dir=True,
    
    num_train_epochs = 2,

    eval_strategy = "epoch",

    gradient_accumulation_steps = 1,
    eval_accumulation_steps = 1,

    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    
    learning_rate = 2e-5,
    weight_decay = 0.01,
    
    logging_first_step=True,
    save_strategy = "steps",
    save_steps=100,                      # Save checkpoints every 100 steps
    save_total_limit=5,

    fp16=True,
)

In [82]:
from transformers import Trainer

ner_trainer = Trainer(
    model = ner_model,
    train_dataset = tokenized_lenerNER["train"],
    eval_dataset = tokenized_lenerNER["validation"],
    data_collator = ner_collator,
    compute_metrics = compute_metrics_ner,
    args = ner_training_args
)

In [83]:
ner_trainer.train()

/share/sw/ai/pytorch/2.5.1/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,4.004500,nan,0.709213,0.812981,0.757560,0.975295
2,4.004500,nan,0.756851,0.835534,0.794248,0.977822


/share/sw/ai/pytorch/2.5.1/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/share/sw/ai/pytorch/2.5.1/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=124, training_loss=0.1392117084995393, metrics={'train_runtime': 68.8142, 'train_samples_per_second': 227.511, 'train_steps_per_second': 1.802, 'total_flos': 3590367998315592.0, 'train_loss': 0.1392117084995393, 'epoch': 2.0})

### Test NER Task

In [84]:
modern_ner_name = f"trained/NER/{model_name}"

In [85]:
import pandas as pd

def tag_sentence(text:str):

    # convert our text to a  tokenized sequence
    inputs = tokenizer(text, truncation=True, return_tensors="pt", return_token_type_ids=False).to("cuda:0")

    # get outputs
    outputs = ner_model(**inputs)

    # convert to probabilities with softmax
    probs = outputs[0][0].softmax(1)

    # get the tags with the highest probability
    word_tags = [(tokenizer.decode(inputs['input_ids'][0][i].item()), id2label[tagid.item()]) 
                  for i, tagid in enumerate (probs.argmax(axis=1))]

    return pd.DataFrame(word_tags, columns=['word', 'tag'])

In [86]:
sample = "O senador de São Paulo decidiu faltar a apelação, movida pelo Supremo Tribunal Federal."

tag_sentence(sample)

,word,tag
0,[CLS],O
1,,O
2,o,O
3,senador,O
4,de,O
5,são,B-LOCAL
6,paulo,I-LOCAL
7,decidiu,O
8,falta,O
9,r,O


Finally, evaluate the fine tuned model in the test dataset

In [87]:
ner_trainer.evaluate(eval_dataset=tokenized_lenerNER["test"])

/share/sw/ai/pytorch/2.5.1/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'eval_loss': 0.044777218252420425,
 'eval_precision': 0.8151057401812689,
 'eval_recall': 0.8910171730515192,
 'eval_f1': 0.8513726727674346,
 'eval_accuracy': 0.9872004068744834,
 'eval_runtime': 4.507,
 'eval_samples_per_second': 308.407,
 'eval_steps_per_second': 2.441,
 'epoch': 2.0}